# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
filepath = "output_data/cities.csv"
citiesDF = pd.read_csv(filepath).drop('Unnamed: 0', axis=1) 
citiesDF.head()

,City,lat,long,Max Temp (F),Cloudiness,Humidity,Wind Speed
0,Vaini,-21.2000,-175.2000,84.4,40,84,5.66
1,Birin,26.8865,88.0718,42.4,50,86,1.30
2,Sobolevo,54.4299,31.8996,27.3,100,96,3.29
3,Mar del Plata,-38.0023,-57.5575,74.1,20,90,10.73
4,Touros,-5.1989,-35.4608,83.3,30,74,5.64


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [66]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure(zoom_level=2, center=[0,0])
heat_layer = gmaps.heatmap_layer(citiesDF[["lat","long"]], weights=citiesDF['Humidity'], max_intensity=citiesDF['Humidity'].max())
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# narrow down the cities...
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
conditions_df = (citiesDF['Max Temp (F)'] < 80) & (citiesDF['Max Temp (F)'] > 70) & (citiesDF['Wind Speed'] < 10) & (citiesDF['Cloudiness'] == 0)
hotel_df = citiesDF.loc[conditions_df]
# how many rows are there? don't want to make too many api calls lol
print(hotel_df.shape[0])

18


In [42]:
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,lat,long,Max Temp (F),Cloudiness,Humidity,Wind Speed,Hotel Name
0,Sunrise Manor,36.2111,-115.0731,75.8,0,20,4.12,Comfort Inn & Suites Las Vegas - Nellis
1,Bonthe,7.5264,-12.5050,77.2,0,91,1.22,Bonthe Holiday Village
2,Parkes,-33.1333,148.1833,75.7,0,37,2.85,Station Motel
3,Nueva Helvecia,-34.3000,-57.2333,73.6,0,48,3.79,La Posada de Ofelia
4,Houston,29.7633,-95.3633,71.8,0,35,4.12,"Hotel ICON, Autograph Collection"
5,Monterey,36.2505,-121.2516,78.1,0,23,0.22,NaN
6,Half Moon Bay,37.4636,-122.4286,79.0,0,39,6.17,NaN
7,Eureka,40.8021,-124.1637,76.1,0,54,1.25,NaN
8,Busselton,-33.6500,115.3333,75.1,0,44,1.48,NaN
9,Takoradze,4.8845,-1.7554,78.9,0,89,3.44,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
request_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [47]:
for index, row in hotel_df.iterrows():
    lat = row['lat']
    long = row['long']
    params['location'] = f'{lat}, {long}'
    try:
        response = requests.get(request_url, params=params).json()
        hotel_name = response['results'][0]['name']
        hotel_df.at[index, 'Hotel Name'] = hotel_name
    except:
        pass
hotel_df

,City,lat,long,Max Temp (F),Cloudiness,Humidity,Wind Speed,Hotel Name
0,Sunrise Manor,36.2111,-115.0731,75.8,0,20,4.12,Comfort Inn & Suites Las Vegas - Nellis
1,Bonthe,7.5264,-12.5050,77.2,0,91,1.22,Bonthe Holiday Village
2,Parkes,-33.1333,148.1833,75.7,0,37,2.85,Station Motel
3,Nueva Helvecia,-34.3000,-57.2333,73.6,0,48,3.79,La Posada de Ofelia
4,Houston,29.7633,-95.3633,71.8,0,35,4.12,"Hotel ICON, Autograph Collection"
5,Monterey,36.2505,-121.2516,78.1,0,23,0.22,NaN
6,Half Moon Bay,37.4636,-122.4286,79.0,0,39,6.17,"The Ritz-Carlton, Half Moon Bay"
7,Eureka,40.8021,-124.1637,76.1,0,54,1.25,Eureka Inn Trademark By Wyndham
8,Busselton,-33.6500,115.3333,75.1,0,44,1.48,Observatory Guest House
9,Takoradze,4.8845,-1.7554,78.9,0,89,3.44,Raybow International Hotel


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [67]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))